### Formatting labels on plots.

In Lets-Plot you can apply a formatting to:

- axis break values.
- labels displayed by `geomText()`.
- tooltip text.
- facet labels.

Using format string you can format values of numeric and date-time types.

In addition, you can use a *string template*.

In *string template* the value's format string is surrounded by curly braces: `"... {.2f} ..."`.

An empty placeholder `{}` is also allowed. In this case a default string representation will be shown. This is also applicable to categorical values.

To learn more about formatting templates see: [Formatting](https://github.com/JetBrains/lets-plot-kotlin/blob/master/docs/formats.md).

In [ ]:
%useLatestDescriptors
%use lets-plot
%use krangl

### The US Unemployment Rates 2000-2016

In [ ]:
var economics = DataFrame.readCSV("https://vincentarelbundock.github.io/Rdatasets/csv/ggplot2/economics.csv")

In [ ]:
import java.text.SimpleDateFormat
val yyyyMmDd = SimpleDateFormat("yyyy-MM-dd")
economics = economics.addColumn("date") { it["date"].map<String> {yyyyMmDd.parse(it)}}
economics.head(3)

In [ ]:
import java.util.Date
val startDate = yyyyMmDd.parse("2000-01-01")
economics = economics.filterByRow { (it["date"] as Date).compareTo(startDate) >= 0 }

In [ ]:
val p = letsPlot(economics.toMap()){x="date"; y="uempmed"} + 
    geomLine() +
    ylab("unemployment rate") +
    ggsize(900, 400)

#### Default plot (no formatting)

In [ ]:
p + scaleXDateTime()

#### Apply formatting to X and Y axis labels.

Use the `format` parameter in `scaleXxx()`.

Note that the text in tooltips is now also formatted.

In [ ]:
p + scaleXDateTime(format="%b %Y") + scaleYContinuous(format="{} %")

#### Format axis labels for breaks specified manually


In [ ]:
val breaks = listOf(
    yyyyMmDd.parse("2001-01-01"), 
    yyyyMmDd.parse("2016-01-01")
)

p + scaleXDateTime(format="%b %Y", breaks=breaks) + scaleYContinuous(format="{} %")

#### Configure tooltip's text and location.

In [ ]:
letsPlot(economics.toMap()){x="date"; y="uempmed"} + 
 geomLine(tooltips=layerTooltips()
        .line("Unemployment rate:|^y")
        .color("black")
        .anchor("top_center")
        .minWidth(170)) + 
 scaleXDateTime(format="%b %Y") +
 scaleYContinuous(format="{} %") +
 ylab("unemployment rate") +
 ggsize(900, 400)

#### Format value shown in tooltip.

In [ ]:
letsPlot(economics.toMap()){x="date"; y="uempmed"} +
 geomLine(tooltips=layerTooltips()
               .line("@uempmed % in @date")
               .format("date", "%B %Y")
               .color("black")
               .anchor("top_left")
               .minWidth(170)) +
 scaleXDateTime() +
 scaleYContinuous() +
 ylab("unemployment rate") +
 ggsize(900, 400)

#### Add the unemployment rate mean.

The `geomText` label is formatted using the `labelFormat` parameter.

In [ ]:
val unemploymentMean = economics["uempmed"].mean()


letsPlot(economics.toMap()){x="date"; y="uempmed"} + 
 geomLine(tooltips=layerTooltips()
        .line("Unemployment rate:|^y")
        .color("black")
        .anchor("top_center")
        .minWidth(170)) + 
 geomHLine(yintercept=unemploymentMean, color="red", linetype="dashed", tooltips=tooltipsNone) +
 geomText(label=unemploymentMean.toString(), 
           labelFormat="{.2f} %",
           x=startDate.toInstant().toEpochMilli(), y=unemploymentMean!! + 0.5, 
           color="red") +
 scaleXDateTime(format="%b %Y") + 
 scaleYContinuous(format="{} %") + 
 ylab("unemployment rate") +
 ggtitle("The US Unemployment Rates 2000-2016.") +
 ggsize(900, 400)